In [10]:
import pandas as pd
pd.options.plotting.backend = "plotly" #interactive plots will be useful in this context
import plotly.express as px
import numpy as np
import gensim
from sentence_transformers import SentenceTransformer, util
import pickle
import torch
from sklearn.cluster import KMeans
from sklearn import metrics
from sklearn.metrics import pairwise_distances
from sklearn.metrics import silhouette_samples, silhouette_score
import umap
from datetime import datetime
from sklearn.cluster import DBSCAN
import dtale

In [2]:
df = pd.read_csv('../datasets/df_full.csv')

In [38]:
row_num = range(len(df))
df['row_num']=row_num

In [4]:
def preprocess_for_bert(string):
    string = string.strip("\n.’:")
    string = string.strip("’")
    string = string.strip("\\n")
    string = string.replace("/"," ")
    return(string)

In [39]:
def pivot(df):#Pivots df of vacancy texts into a df of job requirements retrieved from 'list_elements' in vacancy df
    #model = SentenceTransformer('paraphrase-MiniLM-L6-v2')
    columns = df.columns.values.tolist()
    df_r = pd.DataFrame(columns = ['requirement_raw','requirement_tokenized']+columns)
    for index, row in df.iterrows():
        string = str(row['list_elements'])
        remove = ['\,','[',']','\\n']
        for char in remove:
            string.replace(char, '')
        string = string.strip('\n')
        splitted = string.split('\'') #list of requirements is stored as string so split on '
        list = []
        for item in splitted:
            if len(item)>2:
                requirement_raw = item
                requirement_tokenized = preprocess_for_bert(item)
                requirement_tokenized = gensim.utils.simple_preprocess(requirement_tokenized)
                row_list = [requirement_raw,requirement_tokenized]
                for col in columns:
                    row_list = row_list+[df.loc[index,col]]          
                df_r.loc[len(df_r)] = row_list
        
    requirements = []
    for index, row in df_r.iterrows():
        requirements.append(preprocess_for_bert(row['requirement_raw']))
    return df_r
df_r = pivot(df)

In [40]:
len(df_r)

21509

In [41]:
df_r.head()

,requirement_raw,requirement_tokenized,Unnamed: 0,dt,url,title,location,country,full_text,list_elements,row_num
0,Collaborate with stakeholders and other engine...,"[collaborate, with, stakeholders, and, other, ...",0,2021-08-08 21:01:03.303868,https://se.indeed.com/viewjob?jk=b0669075c820856d,Data Scientist,Jobba hemifrån,se,"About us\n\nHere at Mavenoid, we are building ...",['Collaborate with stakeholders and other engi...,0
1,\nApply the right tools for the job and solve ...,"[apply, the, right, tools, for, the, job, and,...",0,2021-08-08 21:01:03.303868,https://se.indeed.com/viewjob?jk=b0669075c820856d,Data Scientist,Jobba hemifrån,se,"About us\n\nHere at Mavenoid, we are building ...",['Collaborate with stakeholders and other engi...,0
2,\nDevelop knowledge representations and virtua...,"[develop, knowledge, representations, and, vir...",0,2021-08-08 21:01:03.303868,https://se.indeed.com/viewjob?jk=b0669075c820856d,Data Scientist,Jobba hemifrån,se,"About us\n\nHere at Mavenoid, we are building ...",['Collaborate with stakeholders and other engi...,0
3,\nProvide model explanations and apply structu...,"[provide, model, explanations, and, apply, str...",0,2021-08-08 21:01:03.303868,https://se.indeed.com/viewjob?jk=b0669075c820856d,Data Scientist,Jobba hemifrån,se,"About us\n\nHere at Mavenoid, we are building ...",['Collaborate with stakeholders and other engi...,0
4,\nLearn desired behavior from examples using c...,"[learn, desired, behavior, from, examples, usi...",0,2021-08-08 21:01:03.303868,https://se.indeed.com/viewjob?jk=b0669075c820856d,Data Scientist,Jobba hemifrån,se,"About us\n\nHere at Mavenoid, we are building ...",['Collaborate with stakeholders and other engi...,0


In [47]:
l_r = []
for i, r in df_r.iterrows():
    if r['row_num'] not in l_r:
        l_r.append(r['row_num'])
len(l_r)

1103

In [48]:
l = []
for i, r in df.iterrows():
    if r['row_num'] not in l:
        l.append(r['row_num'])
len(l)


2633

In [49]:
for j in l:
    if j not in l_r:
        print(df.iloc[l.index(j)])
        print('------')

Unnamed: 0                                                       6
dt                                      2021-08-08 21:01:33.159440
url              https://se.indeed.com/viewjob?jk=514330948751b10c
title                                              Data Scientist 
location                                         111 22 Stockholm 
country                                                         se
full_text        We are looking for another skilled and driven ...
list_elements                                                   []
row_num                                                          6
Name: 6, dtype: object
------
Unnamed: 0                                                       7
dt                                      2021-08-08 21:01:39.542159
url              https://se.indeed.com/viewjob?jk=9926cbcdcc05c9be
title                                              Data Scientist 
location                                         111 37 Stockholm 
country                         

Name: 796, dtype: object
------
Unnamed: 0                                                    1071
dt                                                             NaN
url              https://se.indeed.com/viewjob?from=appsharedro...
title                                DATA ENGINEER/DATA SCIENTIST 
location                                                Hägersten 
country                                                         se
full_text        In the role as Engineer/Data Scientist you are...
list_elements                                                   []
row_num                                                        797
Name: 797, dtype: object
------
Unnamed: 0                                                    1086
dt                                                             NaN
url              https://es.indeed.com/viewjob?from=appsharedro...
title                                              Data Scientist 
location                           Barcelona, Barcelona provincia

Unnamed: 0                                                    8395
dt                                       2019-09-05 03:12:55 +0000
url              https://job-openings.monster.com/lead-data-sci...
title                       Lead Data Scientist - Machine Learning
location                                               Creve coeur
country                                                         us
full_text        Lead Data Scientist - Machine Learning Lead Da...
list_elements                                                   []
row_num                                                       1134
Name: 1134, dtype: object
------
Unnamed: 0                                                    8396
dt                                       2019-09-05 02:27:30 +0000
url              https://www.careerbuilder.com/job/J3Q1W5767PGK...
title                           Data Scientist Research Programmer
location                                              Santa monica
country                      

Name: 1202, dtype: object
------
Unnamed: 0                                                    8464
dt                                       2019-09-06 04:14:16 +0000
url              https://job-openings.monster.com/senior-data-s...
title                            Senior Data Scientist, Healthcare
location                                                Santa cruz
country                                                         us
full_text        Senior Data Scientist, Healthcare \n If you ar...
list_elements                                                   []
row_num                                                       1203
Name: 1203, dtype: object
------
Unnamed: 0                                                    8465
dt                                       2019-09-06 04:13:17 +0000
url              https://job-openings.monster.com/3-senior-data...
title            (3) Senior Data Scientist - (Machine Learning,...
location                                                 Blue b

Name: 1264, dtype: object
------
Unnamed: 0                                                    8526
dt                                       2019-09-06 04:12:59 +0000
url              https://job-openings.monster.com/data-scientis...
title                                               Data Scientist
location                                          Maryland heights
country                                                         us
full_text        Kforce has a client in search of a Data Scient...
list_elements                                                   []
row_num                                                       1265
Name: 1265, dtype: object
------
Unnamed: 0                                                    8527
dt                                       2019-09-06 04:07:48 +0000
url              https://job-openings.monster.com/data-scientis...
title                                               Data Scientist
location                                                    Res

...

Name: 2299, dtype: object
------
Unnamed: 0                                                    9639
dt                                       2019-09-25 06:20:19 +0000
url              https://www.theladders.com/job/senior-data-sci...
title            Senior Data Scientist, Product Analytics in Sa...
location                                             San francisco
country                                                         us
full_text        The Patreon Data Science team makes it easy to...
list_elements                                                   []
row_num                                                       2300
Name: 2300, dtype: object
------
Unnamed: 0                                                    9640
dt                                       2019-09-25 06:16:14 +0000
url              https://www.theladders.com/job/principal-data-...
title                     Principal Data Scientist in Bellevue, WA
location                                                  Belle

Name: 2365, dtype: object
------
Unnamed: 0                                                    9706
dt                                       2019-09-25 18:22:40 +0000
url              https://job-openings.monster.com/senior-data-s...
title                                        Senior Data Scientist
location                                                  Pasadena
country                                                         us
full_text        Job Summary   \nAs a Senior Data Scientist at ...
list_elements                                                   []
row_num                                                       2366
Name: 2366, dtype: object
------
Unnamed: 0                                                    9707
dt                                       2019-09-26 01:16:09 +0000
url              https://job-openings.monster.com/data-scientis...
title                                          Data Scientist, Mid
location                                                       

------
Unnamed: 0                                                    9777
dt                                       2019-09-27 05:20:54 +0000
url              https://www.theladders.com/job/data-scientist-...
title                           Data Scientist I in Chesapeake, VA
location                                                Chesapeake
country                                                         us
full_text        Purpose of Job Uses advanced techniques that i...
list_elements                                                   []
row_num                                                       2433
Name: 2433, dtype: object
------
Unnamed: 0                                                    9779
dt                                       2019-09-27 05:22:22 +0000
url              https://www.theladders.com/job/sr-data-scienti...
title                          Sr. Data Scientist in Palo Alto, CA
location                                                 Palo alto
country               

Name: 2492, dtype: object
------
Unnamed: 0                                                    9839
dt                                       2019-09-28 00:21:36 +0000
url              https://www.higheredjobs.com/admin/details.cfm...
title                                               Data Scientist
location                                                    Irvine
country                                                         us
full_text        Under the direction of the Director of Enrollm...
list_elements                                                   []
row_num                                                       2493
Name: 2493, dtype: object
------
Unnamed: 0                                                    9840
dt                                       2019-09-28 00:47:34 +0000
url              https://www.higheredjobs.com/admin/details.cfm...
title                           Data Architect Lead/Data Scientist
location                                                 Ann ar

Unnamed: 0                                                    9915
dt                                       2019-09-29 06:34:26 +0000
url              https://www.theladders.com/job/sr-data-scienti...
title                          Sr Data Scientist in Alpharetta, GA
location                                                Alpharetta
country                                                         us
full_text        The Senior Data Scientist position exists to s...
list_elements                                                   []
row_num                                                       2562
Name: 2562, dtype: object
------
Unnamed: 0                                                    9916
dt                                       2019-09-29 06:53:20 +0000
url              https://www.theladders.com/job/director-data-s...
title                       Director, Data Scientist in Toledo, OH
location                                                    Toledo
country                      

In [11]:
dtale.show(df)